In [150]:
import random

from deap import base
from deap import creator
from deap import tools

from PIL import Image, ImageChops
import numpy as np
import math, operator

In [166]:
from deap import base, creator, tools, algorithms
from random import randint, random, gauss
from PIL import Image, ImageDraw
from functools import partial
from math import sqrt
import numpy


PIC = Image.open('template.jpg')
SIZE = 100
NUMBER_OF_TRIANGLES = 100
POPULATION = 75
NGEN = 10000
POLY = 3
VALUES_PER_GENE = 13

NUM_OF_DRAW_OPTIONS = len(["circle","triangle","line","rectangle"])
MAX_CIR_RADIUS = int(SIZE * .1)
LINE_LENGTH = int(SIZE * .15)
LINE_WIDTH = int(SIZE*.01)

MUTATION_NUM_TO_RANDOM = 5

def computeSimilarity(allInds):
    numpyInds = numpy.array(allInds)
    groupedGenes = np.reshape(numpyInds.ravel(order='F'),(NUMBER_OF_TRIANGLES*VALUES_PER_GENE,POPULATION))
    numpy.ndarray.sort(groupedGenes,axis=1)
    
    diffs = numpy.apply_along_axis(np.diff,1,groupedGenes)
    
    multipliers = [i * (POPULATION-i) for i in range(POPULATION)]
    multipliers = multipliers[1:]
    
    scaledDiffs = numpy.apply_along_axis(lambda x: x*multipliers,1,diffs)
    averageDiffInGene = numpy.apply_along_axis(numpy.mean,1,diffs)
    
    return 1-numpy.mean(averageDiffInGene)


def gen_one_triangle():
    return [random() for i in range(VALUES_PER_GENE)]


def drawImage(triangles):
    im = Image.new('RGB', (SIZE, SIZE), (255, 255, 255))
    for tri in triangles:
        mask = Image.new('RGBA', (SIZE, SIZE))
        draw = ImageDraw.Draw(mask)
        
        drawShape = int(tri[0] * NUM_OF_DRAW_OPTIONS)
        x1 = int(tri[1] * SIZE)
        y1 = int(tri[2] * SIZE)
        x2 = int(tri[3] * SIZE)
        y2 = int(tri[4] * SIZE)
        x3 = int(tri[5] * SIZE)
        y3 = int(tri[6] * SIZE)
        r = int(tri[7] * 255)
        g = int(tri[8] * 255)
        b = int(tri[9] * 255)
        alpha = int(tri[10] * 40)
        radius = int(tri[11] * MAX_CIR_RADIUS)
        direction = math.radians(int(tri[12]*360))
        
        fillColor = (r,g,b,alpha)
        
        #triangle
        if drawShape == 0:
            triangle = ((x1,y1),(x2,y2),(x3,y3))
            fillColor = (r,g,b,alpha)
            draw.polygon(triangle, fill=fillColor)

        #circle
        if drawShape == 1:
            lowerX,upperX = x1-radius,x1+radius
            lowerY,upperY = y1-radius,y1+radius
            fillColor = (r,g,b,alpha)
            draw.ellipse((lowerX, lowerY, upperX, upperY), fill = fillColor)

        #line
        if drawShape == 2:
            x2 = x1 + math.cos(direction) * LINE_LENGTH
            y2 = y1 + math.sin(direction) * LINE_LENGTH
            draw.line([x1,y1,x2,y2],width=LINE_WIDTH,fill=fillColor)
        
        #rectangle
        if drawShape == 3:
             draw.rectangle([x1,y1,x2,y2],fill=fillColor)
        
        im.paste(mask, mask=mask)
        del mask, draw
    return im


def evaluate(im1, t2):
    im2 = drawImage(t2)

    h = ImageChops.difference(im1,im2).histogram()
    length = int(len(h)/3)

    r = h[length*0:length*1]
    g = h[length*1:length*2]
    b = h[length*2:length*3]

    h = [sum(x) for x in zip(r,g,b)]
    err = math.sqrt(sum(h*(i**2) for i, h in enumerate(h)) / float(SIZE * SIZE * 3)) / 256
    return 1 - err,



def mutate(triangles):
    for i in range(MUTATION_NUM_TO_RANDOM):
        triangleToMutate = triangles[randint(0,len(triangles)-1)]
        valueIndexToMutate = randint(0,len(triangleToMutate)-1)
        triangleToMutate[valueIndexToMutate] = random()
        
    return triangles,

creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

toolbox = base.Toolbox()
toolbox.register("attr", gen_one_triangle)
toolbox.register("individual", tools.initRepeat,
                 creator.Individual, toolbox.attr, NUMBER_OF_TRIANGLES)
toolbox.register("population", tools.initRepeat,
                 list, toolbox.individual)

toolbox.register("evaluate", partial(evaluate, PIC))
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)


def main():
    pop = toolbox.population(n=POPULATION)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_sim = tools.Statistics(lambda ind: ind)
    stats_fit.register("std", numpy.std)
    stats_fit.register("max", numpy.max)
    stats_fit.register("avg", numpy.mean)
    stats_fit.register("min", numpy.min)
    stats_sim.register("similarity",computeSimilarity)

    mstats = tools.MultiStatistics(fitness=stats_fit, similarity=stats_sim)

    log = ""
    try:
        pop, log = algorithms.eaSimple(
            pop, toolbox, cxpb=0.5, mutpb=0.1, ngen=NGEN, stats=mstats,
            halloffame=hof, verbose=True)
    finally:
        open('stats.txt', 'w').write(str(log))
        open('result.txt', 'w').write(repr(hof[0]))
        drawImage(hof[0]).save('result.png')
        
#if __name__ == '__main__':
#    main()

C:\Users\Joe\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Joe\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


   	      	                     fitness                     	similarity
   	      	-------------------------------------------------	----------
gen	nevals	avg     	max     	min    	std      	similarity
0  	50    	0.526937	0.571686	0.46785	0.0210182	0.98038   
1  	26    	0.543282	0.588842	0.511371	0.0144829	0.981211  
2  	24    	0.550626	0.580935	0.51221 	0.0141329	0.982201  
3  	32    	0.564043	0.593539	0.529137	0.0120071	0.98343   
4  	28    	0.577002	0.617936	0.534314	0.0166715	0.98518   
5  	20    	0.591455	0.625856	0.56651 	0.0118271	0.986603  
6  	32    	0.600674	0.631497	0.562144	0.0160888	0.987285  
7  	25    	0.616675	0.632604	0.588077	0.0121755	0.989726  
8  	27    	0.625981	0.639878	0.600371	0.00803883	0.990858  
9  	22    	0.63216 	0.646261	0.619011	0.00468416	0.991878  
10 	29    	0.635418	0.649939	0.62356 	0.00473117	0.992299  


In [142]:
import cProfile
cProfile.run("main()")

   	      	                         fitness                          	similarity
   	      	----------------------------------------------------------	----------
gen	nevals	avg     	max     	min     	std       	similarity
0  	100   	0.517844	0.539783	0.501597	0.00881915	0.990099  
1  	56    	0.525941	0.54178 	0.50681 	0.00715156	0.990293  
2  	60    	0.530689	0.546347	0.513672	0.00575498	0.990546  
3  	73    	0.534946	0.547324	0.521813	0.00500709	0.990699  
4  	53    	0.53839 	0.553623	0.528654	0.00437188	0.990979  
5  	52    	0.54231 	0.554052	0.532489	0.00466124	0.991398  
         6351714 function calls (5671262 primitive calls) in 21.377 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      600    0.001    0.000    0.003    0.000 <ipython-input-141-a8f175078fd3>:101(<lambda>)
      600    0.000    0.000    0.000    0.000 <ipython-input-141-a8f175078fd3>:102(<lambda>)
        6    0.099    0.016    1.049    0.175 <ipytho